In [1]:
#installing the package for connecting with psotgreSQL
!pip install psycopg2

You should consider upgrading via the 'c:\users\new\anaconda3\python.exe -m pip install --upgrade pip' command.


In [94]:
# function for connecting postgreSQL and creating database for Nifty_stock_data

import psycopg2
def create_database():   
    try:
        conn =psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=root")
    except psycopg2.Error as e:
        print("Database connection failed...")
        print(e)
        
    try:
        curr = conn.cursor()
    except psycopg2.Error as e:
        print("cursor connection is failed")
        print(e) 
    
    conn.set_session(autocommit=True)
    curr.execute("Drop database Nifty_stockdata")
    curr.execute("Create database Nifty_stockdata")
    
    conn.close()
    
    try:
        conn =psycopg2.connect("host=127.0.0.1 dbname=nifty_stockdata user=postgres password=root")
    except psycopg2.Error as e:
        print("Nifty_stockdata Database connection failed...")
        print(e)
    
    curr = conn.cursor()
    
    return conn,curr
    

In [100]:
# to close the connection
try:
    conn.close()
except psycopg2.Error as e:
    print("close connection is failed")
    print(e)

In [4]:
# method to create and drop the table
def create_table(conn,cur):
    for query in create_table_queries:
        curr.execute(query)
        conn.commit()
        
def drop_table(conn,cur):
    for query in drop_table_queries:
        curr.execute(query)
        conn.commit()

In [101]:
# connection is created
conn,curr = create_database()

In [103]:
#loading the input data files via pandas 
import pandas as pd
stock_metadata = pd.read_csv("C:\\Users\\NEW\\Nifty_50_data\\stock_metadata.csv")
HDFC_BANK = pd.read_csv("C:\\Users\\NEW\\Nifty_50_data\\HDFCBANK.csv")

In [104]:
stock_metadata.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finserv Ltd.,FINANCIAL SERVICES,BAJAJFINSV,EQ,INE918I01018


In [105]:
# pre-processing steps for data files
stock_metadata = stock_metadata[['Company Name','Industry','Symbol','ISIN Code']]
HDFC_BANK = HDFC_BANK.fillna(0)
HDFC_BANK = HDFC_BANK[['Date','Symbol','Prev Close','Open','High','Low','Last','Close','VWAP','Volume','Turnover','Trades','Deliverable Volume','%Deliverble']]
HDFC_BANK.head()

In [83]:
# create table Query
hdfc_create_table=("""CREATE TABLE IF NOT EXISTS HDFC_BANK (
   Date DATE PRIMARY KEY,
   Symbol varchar,
   PrevClose decimal,Open decimal,High decimal,Low decimal,Last decimal,Close decimal,VWAP int,Volume int,Turnover bigint,Trades int,
   Deliverable_Volume int, Deliverble int)""")

try:
    curr.execute(hdfc_create_table)
except psycopg2.Error as e:
    print(e)
conn.commit()


stock_metadata_query = ("""CREATE TABLE IF NOT EXISTS Stock_metadata (company_name varchar,industry varchar,symbol varchar, isin_code varchar)""")
try:
    curr.execute(stock_metadata_query)
except psycopg2.Error as e:
    print(e)
conn.commit()

In [51]:
# insert data into the postgreSQL

stock_metadata_insert_query = ("""insert into stock_metadata(company_name,industry,symbol,isin_code)values(%s,%s,%s,%s)""")

for i,row in stock_metadata.iterrows():
    curr.execute(stock_metadata_insert_query,list(row))
conn.commit()

hdfcbank_insert_query = ("""insert into hdfc_bank(date,symbol,prevclose,open,high,low,last,close,vwap,volume,turnover,trades,deliverable_Volume,deliverble)values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""")

for i,row in HDFC_BANK.iterrows():
    curr.execute(hdfcbank_insert_query,list(row))
conn.commit()

In [91]:
# create target table for analaysis

hdfc_create_table_target=("""CREATE TABLE IF NOT EXISTS HDFC_BANK_target (
   Date DATE PRIMARY KEY,
   month varchar,
   year varchar,
   Symbol varchar,
   PrevClose decimal,Open decimal,High decimal,Low decimal,Last decimal,Close decimal,VWAP int,Volume int,Turnover bigint,Trades int,
   Deliverable_Volume int, Deliverble int)""")

try:
    curr.execute(hdfc_create_table_target)
except psycopg2.Error as e:
    print(e)
conn.commit()